In [3]:
pip install scikit-learn


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 11.1 MB 7.8 MB/s eta 0:00:01
     |████████████████████████████████| 30.3 MB 342 kB/s eta 0:00:011   |████▉                           | 4.6 MB 9.5 MB/s eta 0:00:03
     |████████████████████████████████| 307 kB 10.3 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


Defines parameters and paths:

IMG_SIZE: image dimensions (GAN input/output)

NUM_SAMPLES: how many images to evaluate

SKETCH_DIR & REAL_COLOR_DIR: where to find input sketches and real color images

MODEL_WEIGHTS: trained generator weights to be loaded

In [6]:
IMG_SIZE = 128
MODEL_PATH = "/Users/kalyanivaidya/AI Fashion Designer/outputs/render_gan_output/generator_final.weights.h5"


In [8]:
def residual_block(x, filters):
    y = layers.Conv2D(filters, 3, padding='same')(x)
    y = layers.LayerNormalization()(y)
    y = layers.ReLU()(y)
    y = layers.Conv2D(filters, 3, padding='same')(y)
    y = layers.LayerNormalization()(y)
    return layers.add([x, y])

def build_generator():
    inp = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 1))
    x = layers.Conv2D(64, 7, padding='same')(inp)
    x = layers.LayerNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(128, 4, strides=2, padding='same')(x)
    x = layers.LayerNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(256, 4, strides=2, padding='same')(x)
    x = layers.LayerNormalization()(x)
    x = layers.ReLU()(x)
    for _ in range(4):
        x = residual_block(x, 256)
    x = layers.Conv2DTranspose(128, 4, strides=2, padding='same')(x)
    x = layers.LayerNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2DTranspose(64, 4, strides=2, padding='same')(x)
    x = layers.LayerNormalization()(x)
    x = layers.ReLU()(x)
    out = layers.Conv2D(3, 7, padding='same', activation='tanh', dtype='float32')(x)
    return models.Model(inp, out)

In [9]:
G = build_generator()
G.load_weights(MODEL_PATH)
print(" Generator loaded.")

 Generator loaded.


In [10]:
NUM_SAMPLES = 100
SKETCH_DIR = "/Users/kalyanivaidya/AI Fashion Designer/data/processed_sketches"
REAL_DIR = "/Users/kalyanivaidya/AI Fashion Designer/data/processed_originals"

sketch_paths = sorted(tf.io.gfile.glob(os.path.join(SKETCH_DIR, "*.jpg")))[:NUM_SAMPLES]
real_paths = sorted(tf.io.gfile.glob(os.path.join(REAL_DIR, "*.jpg")))[:NUM_SAMPLES]

fake_images, real_images = [], []

for s_path in sketch_paths:
    sketch = load_img(s_path, color_mode='grayscale', target_size=(IMG_SIZE, IMG_SIZE))
    sketch = img_to_array(sketch) / 127.5 - 1.0
    sketch = tf.expand_dims(sketch, 0)
    gen_img = G(sketch, training=False)[0].numpy()
    fake_images.append((np.clip((gen_img + 1) * 127.5, 0, 255)).astype(np.uint8))

for r_path in real_paths:
    img = load_img(r_path, target_size=(IMG_SIZE, IMG_SIZE))
    real_images.append(img_to_array(img).astype(np.uint8))

In [11]:
X = np.array(real_images + fake_images) / 255.0
y = np.array([1]*len(real_images) + [0]*len(fake_images))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = models.Sequential([
    layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.GlobalAveragePooling2D(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
clf.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
clf.fit(X_train, y_train, epochs=5, batch_size=16, verbose=1)

Epoch 1/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - accuracy: 0.5899 - loss: 0.6931
Epoch 2/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.7106 - loss: 0.6743
Epoch 3/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.7500 - loss: 0.6409
Epoch 4/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.8504 - loss: 0.5780
Epoch 5/5
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - accuracy: 0.8497 - loss: 0.5199


In [12]:
y_pred = clf.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

print("\nClassification Report: RenderGAN Classifier\n")
print(classification_report(y_test, y_pred, target_names=['Fake', 'Real']))


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step 

Classification Report: RenderGAN Classifier

              precision    recall  f1-score   support

        Fake       0.73      1.00      0.84        19
        Real       1.00      0.67      0.80        21

    accuracy                           0.82        40
   macro avg       0.87      0.83      0.82        40
weighted avg       0.87      0.82      0.82        40

